In [19]:
%pip install "psycopg[binary,pool]" pytz requests



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import json

# download data
data = {}


# returns JSON

url = "https://api.open-meteo.com/v1/forecast?latitude=47.712596&longitude=13.618029&hourly=temperature_2m,cloud_cover,rain,snowfall,snow_depth,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,relative_humidity_2m,dew_point_2m&timezone=Europe%2FBerlin&forecast_days=14"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Automatically parses JSON into a Python dictionary
    print("Downloaded JSON successfully!")
    print(data)
else:
    print(f"Failed to download JSON. Status code: {response.status_code}")


Downloaded JSON successfully!
{'latitude': 47.7, 'longitude': 13.619999, 'generationtime_ms': 0.4627704620361328, 'utc_offset_seconds': 7200, 'timezone': 'Europe/Berlin', 'timezone_abbreviation': 'GMT+2', 'elevation': 482.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'cloud_cover': '%', 'rain': 'mm', 'snowfall': 'cm', 'snow_depth': 'm', 'cloud_cover_low': '%', 'cloud_cover_mid': '%', 'cloud_cover_high': '%', 'visibility': 'm', 'relative_humidity_2m': '%', 'dew_point_2m': '°C'}, 'hourly': {'time': ['2025-07-07T00:00', '2025-07-07T01:00', '2025-07-07T02:00', '2025-07-07T03:00', '2025-07-07T04:00', '2025-07-07T05:00', '2025-07-07T06:00', '2025-07-07T07:00', '2025-07-07T08:00', '2025-07-07T09:00', '2025-07-07T10:00', '2025-07-07T11:00', '2025-07-07T12:00', '2025-07-07T13:00', '2025-07-07T14:00', '2025-07-07T15:00', '2025-07-07T16:00', '2025-07-07T17:00', '2025-07-07T18:00', '2025-07-07T19:00', '2025-07-07T20:00', '2025-07-07T21:00', '2025-07-07T22:00', '2025-07-07T23:00', 

In [21]:
hourly_units = data['hourly_units']
hourly_units


{'time': 'iso8601',
 'temperature_2m': '°C',
 'cloud_cover': '%',
 'rain': 'mm',
 'snowfall': 'cm',
 'snow_depth': 'm',
 'cloud_cover_low': '%',
 'cloud_cover_mid': '%',
 'cloud_cover_high': '%',
 'visibility': 'm',
 'relative_humidity_2m': '%',
 'dew_point_2m': '°C'}

Load CSV into a Python data structure for processing

In [22]:
metrics = list(hourly_units.keys())
metrics

['time',
 'temperature_2m',
 'cloud_cover',
 'rain',
 'snowfall',
 'snow_depth',
 'cloud_cover_low',
 'cloud_cover_mid',
 'cloud_cover_high',
 'visibility',
 'relative_humidity_2m',
 'dew_point_2m']

In [23]:
import pandas as pd

In [24]:
hourly = data["hourly"]
data_frame_setup = {}

for metric in metrics:
    data_frame_setup[metric] = hourly[metric]
    

In [25]:
df = pd.DataFrame(data_frame_setup)

df['time'] = pd.to_datetime(df['time'])



In [26]:
df.describe()

,time,temperature_2m,cloud_cover,rain,snowfall,snow_depth,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,relative_humidity_2m,dew_point_2m
count,336,336.000000,336.000000,336.000000,336.0,336.0,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000
mean,2025-07-13 23:30:00,18.739881,66.758929,0.202381,0.0,0.0,32.738095,49.678571,38.336310,26765.833333,73.589286,13.448512
min,2025-07-07 00:00:00,10.800000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,3640.000000,35.000000,7.900000
25%,2025-07-10 11:45:00,14.700000,41.000000,0.000000,0.0,0.0,0.000000,3.000000,0.000000,24140.000000,64.000000,11.300000
50%,2025-07-13 23:30:00,17.750000,79.000000,0.000000,0.0,0.0,25.500000,50.500000,26.000000,24140.000000,77.000000,13.650000
75%,2025-07-17 11:15:00,22.050000,100.000000,0.000000,0.0,0.0,58.000000,91.000000,72.000000,24140.000000,86.000000,15.400000
max,2025-07-20 23:00:00,30.600000,100.000000,4.000000,0.0,0.0,100.000000,100.000000,100.000000,71760.000000,96.000000,20.300000
std,NaN,5.077608,35.419089,0.611868,0.0,0.0,33.136666,39.566882,39.382131,12684.312813,15.605522,2.863637


In [27]:
from psycopg import connect
from psycopg.rows import dict_row
from psycopg import sql
import os



os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

data_tuples = [
    (
        row['time'],
        row['temperature_2m'],
        row['cloud_cover'],
        row['rain'],
        row['snowfall'],
        row['snow_depth'],
        row['cloud_cover_low'],
        row['cloud_cover_mid'],
        row['cloud_cover_high'],
        row['visibility'],
        row['relative_humidity_2m'],
        row['dew_point_2m'],
    )
    for _, row in df.iterrows()
]

with connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        cur.executemany(
            """
            INSERT INTO weather_weatherdata (
                time, temperature_2m, cloud_cover, rain, snowfall, snow_depth,
                cloud_cover_low, cloud_cover_mid, cloud_cover_high,
                visibility, relative_humidity_2m, dew_point_2m
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (time) DO UPDATE SET
                temperature_2m = EXCLUDED.temperature_2m,
                cloud_cover = EXCLUDED.cloud_cover,
                rain = EXCLUDED.rain,
                snowfall = EXCLUDED.snowfall,
                snow_depth = EXCLUDED.snow_depth,
                cloud_cover_low = EXCLUDED.cloud_cover_low,
                cloud_cover_mid = EXCLUDED.cloud_cover_mid,
                cloud_cover_high = EXCLUDED.cloud_cover_high,
                visibility = EXCLUDED.visibility,
                relative_humidity_2m = EXCLUDED.relative_humidity_2m,
                dew_point_2m = EXCLUDED.dew_point_2m
            """,
            data_tuples
        )
        print("Data inserted successfully.")
        
        

Data inserted successfully.
